<a href="https://colab.research.google.com/github/medicalmom/M12/blob/master/M12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import datetime as dt
import sys
from scipy import stats

We start by importing modules and then we ingest the dataset we would like to work with.

The New York City Open Data set 311 maintains a dataset for motor vehicle crashes
https://data.cityofnewyork.us/resource/h9gi-nx95.csv

This dataset will be used to examine and look for information on the crashes reported. The data set is ingested below. We will do some cleaning, and grouping before looking for patterns. Any findings, we will use this to look for statistical signficance if we find interesteing observations. 

In [0]:
MVC = pd.read_csv("https://data.cityofnewyork.us/resource/h9gi-nx95.csv?$limit=2000000", low_memory=False)

####Set options on rows and columns to allow all to be seen, or limit to 500 for rows.

In [17]:
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', 500) 
MVC.head()

,accident_date,accident_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2019-07-05T00:00:00.000,14:50,NaN,NaN,40.594970,-73.76071,POINT (-73.76071 40.59497),BEACH 28 STREET,SEAGIRT AVENUE,NaN,1.0,0.0,1,0,0,0,0,0,Failure to Yield Right-of-Way,NaN,NaN,NaN,NaN,4165974,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
1,2019-07-26T00:00:00.000,9:45,NaN,NaN,40.819595,-73.92731,POINT (-73.92731 40.819595),EAST 150 STREET,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Oversized Vehicle,Other Vehicular,NaN,NaN,NaN,4177741,Bus,Taxi,NaN,NaN,NaN
2,2019-07-15T00:00:00.000,18:07,BRONX,10468,40.863785,-73.90028,POINT (-73.90028 40.863785),EAST 190 STREET,JEROME AVENUE,NaN,0.0,0.0,0,0,0,0,0,0,Passing Too Closely,Unspecified,NaN,NaN,NaN,4172081,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
3,2019-07-28T00:00:00.000,17:20,NaN,NaN,40.639725,-74.03605,POINT (-74.03605 40.639725),BELT PARKWAY,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Following Too Closely,Unspecified,NaN,NaN,NaN,4178344,Sedan,Sedan,NaN,NaN,NaN
4,2019-07-06T00:00:00.000,2:45,NaN,NaN,40.757877,-73.85952,POINT (-73.85952 40.757877),110 STREET,NORTHERN BOULEVARD,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,4165326,Sedan,NaN,NaN,NaN,NaN


We would like to get some information about the dataset. Below is the information. 

In [18]:
MVC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1609874 entries, 0 to 1609873
Data columns (total 29 columns):
accident_date                    1609874 non-null object
accident_time                    1609874 non-null object
borough                          1121507 non-null object
zip_code                         1121310 non-null object
latitude                         1413214 non-null float64
longitude                        1413214 non-null float64
location                         1413214 non-null object
on_street_name                   1294265 non-null object
off_street_name                  1070431 non-null object
cross_street_name                223444 non-null object
number_of_persons_injured        1609857 non-null float64
number_of_persons_killed         1609843 non-null float64
number_of_pedestrians_injured    1609874 non-null int64
number_of_pedestrians_killed     1609874 non-null int64
number_of_cyclist_injured        1609874 non-null int64
number_of_cyclist_killed        

We would be interested to know how many null values are in some columns. If there are alot of NaN or null values, we could drop these columns.

In [19]:
MVC[MVC['vehicle_type_code_3'].isnull()].head(50)

,accident_date,accident_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2019-07-05T00:00:00.000,14:50,NaN,NaN,40.594970,-73.760710,POINT (-73.76071 40.59497),BEACH 28 STREET,SEAGIRT AVENUE,NaN,1.0,0.0,1,0,0,0,0,0,Failure to Yield Right-of-Way,NaN,NaN,NaN,NaN,4165974,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
1,2019-07-26T00:00:00.000,9:45,NaN,NaN,40.819595,-73.927310,POINT (-73.92731 40.819595),EAST 150 STREET,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Oversized Vehicle,Other Vehicular,NaN,NaN,NaN,4177741,Bus,Taxi,NaN,NaN,NaN
2,2019-07-15T00:00:00.000,18:07,BRONX,10468,40.863785,-73.900280,POINT (-73.90028 40.863785),EAST 190 STREET,JEROME AVENUE,NaN,0.0,0.0,0,0,0,0,0,0,Passing Too Closely,Unspecified,NaN,NaN,NaN,4172081,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
3,2019-07-28T00:00:00.000,17:20,NaN,NaN,40.639725,-74.036050,POINT (-74.03605 40.639725),BELT PARKWAY,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Following Too Closely,Unspecified,NaN,NaN,NaN,4178344,Sedan,Sedan,NaN,NaN,NaN
4,2019-07-06T00:00:00.000,2:45,NaN,NaN,40.757877,-73.859520,POINT (-73.85952 40.757877),110 STREET,NORTHERN BOULEVARD,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,4165326,Sedan,NaN,NaN,NaN,NaN
5,2019-07-28T00:00:00.000,8:00,MANHATTAN,10016,40.743782,-73.973520,POINT (-73.97352 40.743782),EAST 34 STREET,1 AVENUE,NaN,0.0,0.0,0,0,0,0,0,0,Passing or Lane Usage Improper,Unspecified,NaN,NaN,NaN,4178110,Sedan,Sedan,NaN,NaN,NaN
6,2019-07-29T00:00:00.000,17:12,NaN,NaN,40.879253,-73.919960,POINT (-73.91996 40.879253),HENRY HUDSON PARKWAY,NaN,NaN,1.0,0.0,0,0,0,0,1,0,Lost Consciousness,Unspecified,NaN,NaN,NaN,4180143,Sedan,Dump,NaN,NaN,NaN
7,2019-07-11T00:00:00.000,14:00,NaN,NaN,40.633564,-74.015950,POINT (-74.01595 40.633564),66 STREET,7 AVENUE,NaN,1.0,0.0,0,0,0,0,1,0,Following Too Closely,Unspecified,NaN,NaN,NaN,4168412,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
8,2019-07-02T00:00:00.000,16:14,NaN,NaN,NaN,NaN,NaN,MANHATTAN BR UPPER,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Fatigued/Drowsy,NaN,NaN,NaN,NaN,4163602,Sedan,NaN,NaN,NaN,NaN
9,2019-07-20T00:00:00.000,19:08,BRONX,10472,40.830654,-73.881570,POINT (-73.88157 40.830654),NaN,NaN,1288 EVERGREEN AVENUE,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,4174223,Sedan,Sedan,NaN,NaN,NaN


I would like to group by borough and number of persons killed in the crash to see which boroughs have higher numbers of persons killed. 

In [20]:
MVC1 = MVC.groupby(['borough','number_of_persons_killed'])
MVC1.first()

accident_date accident_time  \
borough       number_of_persons_killed                                          
BRONX         0.0                       2019-07-15T00:00:00.000         18:07   
              1.0                       2019-06-16T00:00:00.000          4:55   
              2.0                       2015-03-20T00:00:00.000         18:34   
BROOKLYN      0.0                       2019-07-17T00:00:00.000         20:50   
              1.0                       2019-07-08T00:00:00.000          9:10   
              2.0                       2019-06-08T00:00:00.000          0:07   
              3.0                       2013-03-03T00:00:00.000          0:05   
MANHATTAN     0.0                       2019-07-28T00:00:00.000          8:00   
              1.0                       2019-07-07T00:00:00.000         12:59   
              8.0                       2017-10-31T00:00:00.000         15:08   
QUEENS        0.0                       2019-07-15T00:00:00.000         16:50   
              1.0                       2019-07-28T00:00:00.000         16:36   
              2.0                       2017-06-17T00:00:00.000          0:55   
              3.0                       2015-07-01T00:00:00.000         18:06   
              4.0                       2017-09-18T00:00:00.000          6:17   
              5.0                       2012-07-22T00:00:00.000          3:20   
STATEN ISLAND 0.0                       2019-07-12T00:00:00.000         18:30   
              1.0                       2019-07-23T00:00:00.000         12:00   
              2.0                       2015-10-05T00:00:00.000         23:00   

                                       zip_code   latitude  longitude  \
borough       number_of_persons_killed                                  
BRONX         0.0                         10468  40.863785 -73.900280   
              1.0                         10455  40.811428 -73.900910   
              2.0                         10456  40.838722 -73.913771   
BROOKLYN      0.0                         11221  40.696873 -73.927920   
              1.0                         11218  40.646378 -73.970850   
              2.0                         11234  40.608204 -73.920715   
              3.0                         11249  40.703159 -73.965874   
MANHATTAN     0.0                         10016  40.743782 -73.973520   
              1.0                         10027  40.812103 -73.949760   
              8.0                         10014  40.729046 -74.010730   
QUEENS        0.0                         11385  40.697315 -73.905590   
              1.0                         11385  40.700890 -73.893360   
              2.0                         11423  40.711840 -73.754090   
              3.0                         11364  40.744509 -73.766099   
              4.0                         11354  40.762978 -73.831980   
              5.0                         11418  40.697204 -73.813334   
STATEN ISLAND 0.0                         10304  40.626930 -74.077900   
              1.0                         10310  40.633858 -74.125960   
              2.0                         10312  40.529374 -74.161351   

                                                              location  \
borough       number_of_persons_killed                                   
BRONX         0.0                          POINT (-73.90028 40.863785)   
              1.0                          POINT (-73.90091 40.811428)   
              2.0                       POINT (-73.9137706 40.8387216)   
BROOKLYN      0.0                          POINT (-73.92792 40.696873)   
              1.0                          POINT (-73.97085 40.646378)   
              2.0                         POINT (-73.920715 40.608204)   
              3.0                       POINT (-73.9658741 40.7031587)   
MANHATTAN     0.0                          POINT (-73.97352 40.743782)   
              1.0                          POINT (-73.94976 40.812103)   


Look at this same information as the dataframe.

In [21]:
MVC1.head(30)


,accident_date,accident_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2019-07-05T00:00:00.000,14:50,NaN,NaN,40.594970,-73.760710,POINT (-73.76071 40.59497),BEACH 28 STREET,SEAGIRT AVENUE,NaN,1.0,0.0,1,0,0,0,0,0,Failure to Yield Right-of-Way,NaN,NaN,NaN,NaN,4165974,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
1,2019-07-26T00:00:00.000,9:45,NaN,NaN,40.819595,-73.927310,POINT (-73.92731 40.819595),EAST 150 STREET,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Oversized Vehicle,Other Vehicular,NaN,NaN,NaN,4177741,Bus,Taxi,NaN,NaN,NaN
2,2019-07-15T00:00:00.000,18:07,BRONX,10468,40.863785,-73.900280,POINT (-73.90028 40.863785),EAST 190 STREET,JEROME AVENUE,NaN,0.0,0.0,0,0,0,0,0,0,Passing Too Closely,Unspecified,NaN,NaN,NaN,4172081,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
3,2019-07-28T00:00:00.000,17:20,NaN,NaN,40.639725,-74.036050,POINT (-74.03605 40.639725),BELT PARKWAY,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Following Too Closely,Unspecified,NaN,NaN,NaN,4178344,Sedan,Sedan,NaN,NaN,NaN
4,2019-07-06T00:00:00.000,2:45,NaN,NaN,40.757877,-73.859520,POINT (-73.85952 40.757877),110 STREET,NORTHERN BOULEVARD,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,4165326,Sedan,NaN,NaN,NaN,NaN
5,2019-07-28T00:00:00.000,8:00,MANHATTAN,10016,40.743782,-73.973520,POINT (-73.97352 40.743782),EAST 34 STREET,1 AVENUE,NaN,0.0,0.0,0,0,0,0,0,0,Passing or Lane Usage Improper,Unspecified,NaN,NaN,NaN,4178110,Sedan,Sedan,NaN,NaN,NaN
6,2019-07-29T00:00:00.000,17:12,NaN,NaN,40.879253,-73.919960,POINT (-73.91996 40.879253),HENRY HUDSON PARKWAY,NaN,NaN,1.0,0.0,0,0,0,0,1,0,Lost Consciousness,Unspecified,NaN,NaN,NaN,4180143,Sedan,Dump,NaN,NaN,NaN
7,2019-07-11T00:00:00.000,14:00,NaN,NaN,40.633564,-74.015950,POINT (-74.01595 40.633564),66 STREET,7 AVENUE,NaN,1.0,0.0,0,0,0,0,1,0,Following Too Closely,Unspecified,NaN,NaN,NaN,4168412,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
8,2019-07-02T00:00:00.000,16:14,NaN,NaN,NaN,NaN,NaN,MANHATTAN BR UPPER,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Fatigued/Drowsy,NaN,NaN,NaN,NaN,4163602,Sedan,NaN,NaN,NaN,NaN
9,2019-07-20T00:00:00.000,19:08,BRONX,10472,40.830654,-73.881570,POINT (-73.88157 40.830654),NaN,NaN,1288 EVERGREEN AVENUE,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,4174223,Sedan,Sedan,NaN,NaN,NaN


I would like to know what numbers are associated with deaths for crashes in Queens and look at this information. 

In [24]:
MVC1.get_value('QUEENS')


AttributeError: ignored

I would like to drop a few columns that have quite a few NaN. 

In [28]:
MVC2 = MVC.drop(['contributing_factor_vehicle_3', 'contributing_factor_vehicle_4', 'contributing_factor_vehicle_5'],axis=1)
MVC2

,accident_date,accident_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,collision_id,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2019-07-05T00:00:00.000,14:50,NaN,NaN,40.594970,-73.760710,POINT (-73.76071 40.59497),BEACH 28 STREET,SEAGIRT AVENUE,NaN,1.0,0.0,1,0,0,0,0,0,Failure to Yield Right-of-Way,NaN,4165974,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
1,2019-07-26T00:00:00.000,9:45,NaN,NaN,40.819595,-73.927310,POINT (-73.92731 40.819595),EAST 150 STREET,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Oversized Vehicle,Other Vehicular,4177741,Bus,Taxi,NaN,NaN,NaN
2,2019-07-15T00:00:00.000,18:07,BRONX,10468,40.863785,-73.900280,POINT (-73.90028 40.863785),EAST 190 STREET,JEROME AVENUE,NaN,0.0,0.0,0,0,0,0,0,0,Passing Too Closely,Unspecified,4172081,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
3,2019-07-28T00:00:00.000,17:20,NaN,NaN,40.639725,-74.036050,POINT (-74.03605 40.639725),BELT PARKWAY,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Following Too Closely,Unspecified,4178344,Sedan,Sedan,NaN,NaN,NaN
4,2019-07-06T00:00:00.000,2:45,NaN,NaN,40.757877,-73.859520,POINT (-73.85952 40.757877),110 STREET,NORTHERN BOULEVARD,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,NaN,4165326,Sedan,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1609869,2012-10-21T00:00:00.000,17:17,QUEENS,11432,40.705762,-73.801657,POINT (-73.8016567 40.7057624),89 AVENUE,PARSONS BOULEVARD,NaN,0.0,0.0,0,0,0,0,0,0,Backing Unsafely,Unspecified,209466,PASSENGER VEHICLE,UNKNOWN,NaN,NaN,NaN
1609870,2012-10-19T00:00:00.000,12:42,STATEN ISLAND,10314,40.584040,-74.158916,POINT (-74.1589161 40.58404),WESTPORT LANE,RICHMOND HILL ROAD,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,294787,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
1609871,2012-10-19T00:00:00.000,22:55,BRONX,10462,40.833614,-73.860851,POINT (-73.860851 40.8336142),HUGH GRANT CIRCLE,METROPOLITAN AVENUE,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,81579,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
1609872,2012-10-18T00:00:00.000,15:40,NaN,NaN,40.760935,-73.731124,POINT (-73.7311243 40.7609347),NaN,NaN,NaN,1.0,0.0,0,0,0,0,1,0,Unspecified,Unspecified,3079084,PASSENGER VEHICLE,LARGE COM VEH(6 OR MORE TIRES),NaN,NaN,NaN


In [0]:
?stats.pearsonr

In [29]:
stats.pearsonr(MVC['borough'], MVC['number_of_persons_killed'])

ValueError: ignored